In [1]:
import tensorflow as tf

In [2]:
# 图的构建
m1 = tf.constant([3,5])
m2 = tf.constant([2,4])
result = tf.add(m1, m2)
# 输出结果并没有输出矩阵相加的结果, 而是输出了一个包含三个属性的Tensor
print(result)

Tensor("Add:0", shape=(2,), dtype=int32)


![图的构建过程](./figs/fig-1.png)

In [3]:
# 第二个阶段为图的执行阶段
sess = tf.Session()
print(sess.run(result))
sess.close()

[5 9]


In [4]:
# 建议使用with 代码块的形式
with tf.Session() as sess:
    res = sess.run([result])
print(res)

[array([5, 9])]


In [5]:
# 如果有可用的GUI环境, 可以通过device指定GPU进行运行
with tf.Session() as sess:
    # 这里没有gpu环境，先屏蔽掉
    #with tf.device('/gpu:2'):
    m1 = tf.constant([3,5])
    m2 = tf.constant([2,4])
    result = tf.add(m1,m2)
    print(sess.run(result))

[5 9]


In [6]:
# 默认会话机制
m1 = tf.constant([3,5])
m2 = tf.constant([2,4])
result = tf.add(m1, m2)
sess = tf.Session()
with sess.as_default():
    print(result.eval())

[5 9]


In [7]:
# 专门为jupyter/IPython等交互环境开发的
m1 = tf.constant([3,5])
m2 = tf.constant([2,4])
result = tf.add(m1, m2)
sess = tf.InteractiveSession()
print(result.eval())

[5 9]


## Tensor介绍

我们可以把Tensor理解成N维矩阵（N维数组）。其中零维张量表示的是一个标量，也就是一个数；一维张量表示的是一个向量，也可以看作是一个一维数组；二维张量表示的是一个矩阵；同理，N维张量也就是N维矩阵。

In [8]:
# 在计算图模型中, 操作间所传递的数据都可以看作是Tensor
a = tf.constant([[2.0, 3.0]], name = 'a')
b = tf.constant([[1.0],[4.0]], name = 'b')
result = tf.matmul(a, b, name = 'mul')
print(result)
# mul_1:0 表示是第一个结果的输出

Tensor("mul:0", shape=(1, 1), dtype=float32)


### 常量、变量及占位符

In [9]:
# constant相关属性
a = tf.constant([2.0,3.0], name='a',shape=(2,0), dtype='float64', verify_shape='true')
print(a)

Tensor("a_1:0", shape=(2, 0), dtype=float64)


Tensorflow还提供了一些常见常量的初始化，如：tf.zeros、tf.ones、tf.fill、tf.linspace、tf.range等，均可以快速初始化一些常量。例如：我们想要快速初始化N维全0的矩阵，我们可以利用tf.zeros进行初始化

In [10]:
a = tf.zeros([2,2], tf.float32)
b = tf.zeros_like(a, optimize=True)
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))

[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]


此外，Tensorflow还可以生成一些随机的张量，方便快速初始化一些随机值。如：tf.random_normal()、tf.truncated_normal()、tf.random_uniform()、tf.random_shuffle()等。

In [11]:
random_num = tf.random_normal([2,3], mean=-1, stddev=4, dtype=tf.float32, seed = None, name='rnum')
with tf.Session() as sess:
    print(sess.run(random_num))

[[  8.261671   -4.7531786   1.3353057]
 [ -0.9768454   2.2488935 -11.763928 ]]


In [12]:
# variable() 的使用
A = tf.Variable(3, name='number')
B = tf.Variable([1,3], name='vector')
C = tf.Variable([[0,1],[2,3]], name='matrix')
D = tf.Variable(tf.zeros([100]), name='zero')
E = tf.Variable(tf.random_normal([2,3], mean=1, stddev=2, dtype=tf.float32))

In [13]:
# 变量初始化
# 初始化全局变量
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
# 初始化变量的子集:
init_subset = tf.variables_initializer([B,C], name='init_subset')
with tf.Session() as sess:
    sess.run(init_subset)
# 初始化单个变量
init_var = tf.Variable(tf.zeros([2,5]))
with tf.Session() as sess:
    sess.run(init_var.initializer)

In [14]:
## 变量保存
tf.reset_default_graph()
var1 = tf.Variable([1,3], name='V1')
var1 = tf.Variable([2,4], name='V2')
# 对全部变量进行初始化
init = tf.global_variables_initializer()
# 调用Saver()存储器方法
saver = tf.train.Saver()
# 执行图模型
with tf.Session() as sess:
    sess.run(init)
    for i, var in enumerate(saver._var_list):
        print('Var {}: {}'.format(i,var))
    # 设置存储路径
    save_path = saver.save(sess, './data/save.ckpt')

Var 0: <tf.Variable 'V1:0' shape=(2,) dtype=int32_ref>
Var 1: <tf.Variable 'V2:0' shape=(2,) dtype=int32_ref>


In [15]:
import os
for file in os.listdir('./data/'):
    if 'save' in file:
        print(file)

save.ckpt.data-00000-of-00001
save.ckpt.index
save.ckpt.index.tempstate16530603378328715372
save.ckpt.index.tempstate639255877349795406
save.ckpt.meta


In [16]:
# 恢复变量
tf.reset_default_graph()
var1 = tf.Variable([0,0], name='V1')
var2 = tf.Variable([0,0], name='V2')
saver = tf.train.Saver()
module_file = tf.train.latest_checkpoint('./data/')
with tf.Session() as sess:
    saver.restore(sess, './data/save.ckpt')
    print('Model Restored.')
    print(sess.run([var1, var2]))

W0806 09:29:02.122171 13344 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Model Restored.
[array([1, 3]), array([2, 4])]


In [17]:
# placeholder 是数据初始化的容器
a = tf.placeholder(tf.float32, shape=[2], name=None)
b = tf.constant([6,4], tf.float32)
c = tf.add(a,b)
with tf.Session() as sess:
    print(sess.run(c, feed_dict = {a:[10,10]}))

[16. 14.]


In [18]:
# fetch, fetch的含义是指可以在一个绘画中同时运行多个op, 可以同时取回多个tensor值
a = tf.constant(5)
b = tf.constant(6)
c = tf.constant(4)
add = tf.add(b,c)
mul = tf.multiply(a, add)
with tf.Session() as sess:
    result = sess.run([mul, add])
    print(result)

[50, 10]
